# Statistical comparison using Bayesian Statistics
___

## Motivation

Bayesian Statistics is the new prefered way of comparing results in Machine Learning, it has many advantages over the traditional frequentist approach.

## Background

Bayesian Statistics is based on [Bayes' Theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem) and we have that the posterior is proportional to the prior times the likelihood:

$\text{posterior} \propto \text{prior} * \text{likelihood} $

the prior being the initial belief about the population and the likelihood the probability of occurrence of each sample in the population.

After calculating the posterior, it holds our entire belief about the population and we can use it to answer questions probabilistically.

## Assumptions

Below we consider the set of live results of a single account as a single population.

Also, it is assumed that accounts did not change models in the past 30 rounds because we cannot know this from other people's accounts.

## Evaluating a single account

The mean of a population is lower than zero?
This is a null hypothesis that we can evaluate using Bayesian Statistics.

And we can use the posterior to answer the following related question:
What's the probability of the mean of a population being greater than zero?

### How many live results should we use to evaluate an account?

The [convergence plot below](#Plots) tries to answer this question.
The plot is read backwards, as we consider an increasing number of points from the most recent results, the mean converges to a value and the 98% high density interval (HDI) converges to a small region.

From this plot we select the number of rounds and we ask, can we reject the null hypothesis?
- For arbitrage:
  - considering 10 rounds, we cannot because zero is inside the HDI;
  - considering 20 rounds, we can reject the null hypothesis.
- For benben11:
  - considering just 6 rounds, we can already reject the null hypothehis.
- For jrb:
  - even with 30 rounds, we cannot reject the null hypothesis.
  
When we can reject the null hypothesis, we also know that the probability of the mean of the population being grater than zero is greater than 99%!
This is because we are performing a two-sided test.
To calculate the 98% HDI, there's 1% above and below it... so when the HDI is above zero, more than 99% of the posterior probability is above zero.

### How an account performance evolves after each round?

The [timeseries plot below](#Plots) tries to answer this question.
After selecting an appropriate number of rounds from the convergence plot, we can see in the timeseries plot how the posterior mean and HDI evolve at each round.

Using 20 rounds as the official leaderboard, we note that:
- For arbitrage, the HDI has been above zero only in the latest 11 rounds;
- For benben11, the latest 10 rounds gave a huge boost to the HDI;
- For hiryuu, the HDI has been steady above zero for the latest 20 rounds;
- For jrb, the HDI has not been above zero for the latest 30 rounds;
- For themicon, the HDI was well above zero for most rounds but it has lowered in the latest 5 rounds.

## Comparing multiple accounts

Is the difference of the mean of two populations equal to zero?
This is a null hypothesis that cannot be evaluated using Bayesian Statistics because the probability of occurrence of a single particular value is, by definition, zero.

Yet, we can use the posterior to answer the following related question:
What's the probability of the mean of a population being greater than the mean of another population?

It is helpful to define a region where we consider two populations as being equivalent, the region of practical equivalence (ROPE).

Below, we consider that two populations with the difference of their means lower than 0.0025 as being equivalent and ignore this region, we are interested in the probability of a population being better than another.

Still, comparing just two populations is very limited, so we use the posterior to calculate the probabilities of each population being better and worse than every other population, building a matrix of probabilities, and then we simply take the means and rank the populations.

We end up with a mean probability of each population being better than all the others.

We show some examples of rankings:
- [Some popular accounts](#Some-popular-accounts);
- [The top 20 accounts](#Top-20-accounts);
- [Nasdaqjockey's accounts](#nasdaqjockey's-accounts), showing that this ranking strategy would work well with [their method to select diagnostic metrics](https://forum.numer.ai/t/does-good-model-diagnostics-correlate-with-tournament-performance/).

## Conclusion

These are statistical tests about the mean of the populations, meaning that an account can still suffer serious burns in upcoming rounds but we get from the results some hints that help us evaluate how many rounds to consider and how to compare models before deciding to replace underperforming models and how to allocate stakes.

## Links

- [Statistical comparison of models using grid search](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_stats.html)
- [Time for a Change:  a Tutorial for Comparing Multiple Classifiers Through Bayesian Analysis](https://www.jmlr.org/papers/volume18/16-305/16-305.pdf)
- [Bayesian Estimation Supersedes the T-Test](https://docs.pymc.io/notebooks/BEST.html)
- [Introduction to Bayesian Stastistics](https://www.amazon.com/Introduction-Bayesian-Statistics-William-Bolstad/dp/1118091566)

In [1]:
import arviz as az
import numpy as np
import pymc3 as pm
import pandas as pd

from scipy import stats
from numerapi import NumerAPI
from tqdm.notebook import tqdm

from bokeh.io import output_notebook, show
from bokeh.models import Span
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [2]:
import logging

logging.getLogger('pymc3').setLevel(logging.CRITICAL)
logging.getLogger('filelock').setLevel(logging.WARNING)
logging.getLogger('numexpr.utils').setLevel(logging.WARNING)

In [3]:
napi = NumerAPI()


def get_metrics(username):
    try:
        df = pd.DataFrame(napi.daily_submissions_performances(username))
    except:
        return None

    results = []
    for name, group in df.groupby('roundNumber'):
        values = group.drop(['correlationWithMetamodel', 'fnc'], axis=1).dropna()
        if values.shape[0] > 0:
            values = values.sort_values('date').tail(1).to_dict('records')[0]
            results.append({
                'round': name,
                'corr': values['correlation'],
                'mmc': values['mmc'],
                'corr+mmc': values['correlation'] + values['mmc'],
            })
    return pd.DataFrame(results)

In [4]:
def get_summary_mean(observed, hdi_prob=0.98):
     with pm.Model():
        prior_mean = pm.Normal('mean', mu=0, sigma=1)
        prior_std = pm.HalfNormal('std', sigma=0.4 / 6)
        likelihood = pm.Normal('likelihood', mu=prior_mean, sigma=prior_std, observed=observed)

        trace = pm.sample(2000, return_inferencedata=True, progressbar=False)
        return az.summary(trace, hdi_prob=hdi_prob).T['mean'].to_dict()

In [5]:
def get_convergence(metrics, metric_name='corr+mmc', n_rounds=30):
    stats = []
    for n in tqdm(list(range(1, n_rounds+1)), leave=False):
        df = metrics.tail(n)

        if len(df) < n:
            continue

        idx = df['round'].values[0]
        observed = df[metric_name].values
        summary = get_summary_mean(observed)
        stats.append({
            'round': idx,
            'mean': summary['mean'],
            'lower': summary['hdi_1%'],
            'higher': summary['hdi_99%'],
        })

    stats = pd.DataFrame(stats)
    return metrics.merge(stats, on='round')

In [6]:
def get_timeseries(metrics, metric_name='corr+mmc', n_rounds=30, n_points=20):
    stats = []
    for n in tqdm(list(range(n_rounds)), leave=False):
        df = metrics.head(len(metrics) - n)
        df = df.tail(n_points)

        if len(df) < 1:
            continue

        idx = df['round'].values[-1]
        observed = df[metric_name].values
        summary = get_summary_mean(observed)
        stats.append({
            'round': idx,
            'mean': summary['mean'],
            'lower': summary['hdi_1%'],
            'higher': summary['hdi_99%'],
        })

    stats = pd.DataFrame(stats)
    return metrics.merge(stats, on='round')

In [7]:
def plot_series(data, title='', metric_name='corr+mmc'):
    plt = figure(width=900, height=300, y_range=(-0.26, 0.26), title=title)

    plt.varea(data['round'], data['lower'], data['higher'], color='gray', alpha=0.1, legend_label='hdi')

    plt.line(data['round'], data['mean'], color='green', line_width=1, legend_label='mean')

    plt.line(data['round'], data[metric_name], color='blue', line_width=1, legend_label='metric')
    plt.circle(data['round'], data[metric_name], color='blue', line_width=1, legend_label='metric')

    if len(data) > 19:
        plt.add_layout(Span(location=data['round'].max() - 19,
                            dimension='height',
                            line_color='gray',
                            line_dash='dashed'))

    if len(data) > 4:
        plt.add_layout(Span(location=data['round'].max() - 3,
                            dimension='height',
                            line_color='gray',
                            line_dash='dashed'))

    plt.legend.location = 'bottom_left'
    plt.legend.click_policy = 'hide'
    return plt

In [8]:
accounts = [
    'arbitrage',
    'benben11',
    'hiryuu',
    'integration_test_7',
    'jrb',
    'nasdaqjockey',
    'niam',
    'sorios',
    'themicon',
    'uuazed',
]

results = {n: get_metrics(n) for n in tqdm(accounts)}

In [9]:
convergences = {k: get_convergence(v) for k, v in tqdm(results.items())}

In [10]:
timeseries = {k: get_timeseries(v) for k, v in tqdm(results.items())}

# Plots

In [11]:
for name in accounts:
    if name in convergences and name in timeseries:
        show(plot_series(convergences[name], title=f'Convergence - {name}'))
        show(plot_series(timeseries[name], title=f'Timeseries - {name}'))

In [12]:
def get_summary_diff(observed, hdi_prob=0.94):
    with pm.Model():
        prior_mean_0 = pm.Normal('mean0', mu=0, sigma=1)
        prior_std_0 = pm.HalfNormal('std0', sigma=0.4 / 6)
        likelihood_0 = pm.Normal('likelihood0', mu=prior_mean_0, sigma=prior_std_0, observed=observed[0])
        
        prior_mean_1 = pm.Normal('mean1', mu=0, sigma=1)
        prior_std_1 = pm.HalfNormal('std1', sigma=0.4 / 6)
        likelihood_1 = pm.Normal('likelihood1', mu=prior_mean_1, sigma=prior_std_1, observed=observed[1])

        mean_diff = pm.Deterministic('mean_diff', prior_mean_0 - prior_mean_1)
        std_diff = pm.Deterministic('std_diff', prior_std_0 - prior_std_1)
        effect_size = pm.Deterministic('effect_size',
                                       mean_diff / np.sqrt((prior_std_0 ** 2 + prior_std_1 ** 2) / 2))

        trace = pm.sample(2000, return_inferencedata=True, progressbar=False)
        return az.summary(trace, hdi_prob=hdi_prob).T['mean_diff'].to_dict()

In [13]:
def compare(series, n_rounds=20, prob=0.94, rope_prob=0.0025):
    summary = get_summary_diff([
        series[0].tail(n_rounds).values,
        series[1].tail(n_rounds).values
    ], hdi_prob=prob)
    posterior = stats.norm(loc=summary['mean'], scale=summary['sd'])
    lower = posterior.cdf(-rope_prob)
    higher = posterior.cdf(rope_prob)
    return 1-higher, lower

In [14]:
def get_probs(results, accounts, metric_name='corr+mmc', **kwargs):
    n_accounts = len(accounts)
    data = np.full((n_accounts, n_accounts), np.nan)
    for i in tqdm(list(range(n_accounts))):
        for j in range(i+1, n_accounts):
            better, worse = compare([results[accounts[i]][metric_name], results[accounts[j]][metric_name]], **kwargs)
            data[i, j] = better
            data[j, i] = worse
    return pd.DataFrame(data, columns=accounts, index=accounts)

In [15]:
accounts = [
    'arbitrage',
    'benben11',
    'hiryuu',
    'integration_test_7',
    'jrb',
    'nasdaqjockey',
    'niam',
    'sorios',
    'themicon',
    'uuazed',
]

results = {n: get_metrics(n) for n in tqdm(accounts)}

In [16]:
probs = get_probs(results, accounts)

In [17]:
probs

,arbitrage,benben11,hiryuu,integration_test_7,jrb,nasdaqjockey,niam,sorios,themicon,uuazed
arbitrage,NaN,3.159335e-03,3.508822e-04,0.636831,0.997906,0.999596,0.967843,0.522152,0.752323,0.232460
benben11,0.990516,NaN,5.497382e-01,0.997505,1.000000,1.000000,0.999949,0.997378,0.999081,0.953382
hiryuu,0.997697,2.940241e-01,NaN,0.999423,1.000000,1.000000,1.000000,0.999816,0.999975,0.953698
integration_test_7,0.197663,7.057045e-04,8.841729e-05,NaN,0.987035,0.995975,0.852094,0.270563,0.554233,0.098236
jrb,0.000453,4.492143e-08,2.876854e-11,0.003661,NaN,0.480061,0.079404,0.000099,0.007996,0.000144
nasdaqjockey,0.000059,2.929088e-08,7.403041e-14,0.000816,0.326355,NaN,0.040059,0.000063,0.002186,0.000075
niam,0.009387,9.807413e-06,5.256983e-09,0.066807,0.830096,0.894350,NaN,0.015130,0.109860,0.005516
sorios,0.270563,6.675333e-04,1.410220e-05,0.522152,0.999229,0.999477,0.946422,NaN,0.691462,0.127902
themicon,0.127902,2.326291e-04,2.002612e-06,0.277291,0.974682,0.990613,0.780158,0.145586,NaN,0.052081
uuazed,0.635386,2.088953e-02,1.632489e-02,0.809213,0.999332,0.999631,0.983207,0.752323,0.886540,NaN


# Some popular accounts

In [18]:
probs.mean(axis=1).rank(ascending=False).sort_values().to_frame('rank').join(
    probs.mean(axis=1).to_frame('mean_prob'))

,rank,mean_prob
benben11,1.0,0.943061
hiryuu,2.0,0.916070
uuazed,3.0,0.678094
arbitrage,4.0,0.568069
sorios,5.0,0.506432
integration_test_7,6.0,0.439621
themicon,7.0,0.372061
niam,8.0,0.214573
jrb,9.0,0.063535
nasdaqjockey,10.0,0.041068


In [19]:
accounts = [x['username'] for x in napi.get_leaderboard(20)]

results = {n: get_metrics(n) for n in tqdm(accounts)}

In [20]:
probs = get_probs(results, accounts, 'corr')

# Top 20 accounts

In [21]:
probs.mean(axis=1).rank(ascending=False).sort_values().to_frame('rank').join(
    probs.mean(axis=1).to_frame('mean_prob'))

,rank,mean_prob
quantyquant,1.0,0.651241
benben11,2.0,0.646935
hb_falcon,3.0,0.638706
hiryuu,4.0,0.497765
labrat,5.0,0.489565
uuazed6,6.0,0.480233
neuralnetwork3,7.0,0.475968
uuazed5,8.0,0.457606
anna13,9.0,0.437460
junyou,10.0,0.377131


In [23]:
accounts = [
    'evolvz',
    'zbrain',
    'nasdaqjockey',
    'nasdaqjockey1',
    'nasdaqjockey2',
    'nasdaqjockey3',
    'nasdaqjockey4',
    'nasdaqjockey5',
    'nasdaqjockey6',
    'nasdaqjockey7',
    'nasdaqjockey8',
    'nasdaqjockey9',
    'nasdaqjockey10',
    'nasdaqjockey11',
    'nasdaqjockey12',
]

results = {n: get_metrics(n) for n in tqdm(accounts)}

In [24]:
probs = get_probs(results, accounts, 'corr', n_rounds=13)

# nasdaqjockey's accounts

In [25]:
probs.mean(axis=1).rank(ascending=False).sort_values().to_frame('rank').join(
    probs.mean(axis=1).to_frame('mean_prob'))

,rank,mean_prob
nasdaqjockey4,1.0,0.832989
nasdaqjockey3,2.0,0.693676
nasdaqjockey9,3.0,0.652473
nasdaqjockey12,4.0,0.520772
evolvz,5.0,0.462582
nasdaqjockey1,6.0,0.459245
nasdaqjockey7,7.0,0.410780
nasdaqjockey10,8.0,0.337432
nasdaqjockey5,9.0,0.322857
nasdaqjockey11,10.0,0.299672
